In [1]:
from ktms import *
from MAInit import *
from MACalc import *
from MAPost import *
import re

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client.adsE_database
collection = db.adsE_collection

In [5]:
def makejson(file):
    atoms = query(file, 'spacom')

    dic = {}

    res = re.match('(.*u\d).*', file)
    barefile = res.group(1) + '.traj'
    bareatoms = query(barefile, 'spacom')
#     print(bareatoms)

    COatoms = query('CO.traj', 'scpacom')
    COene = COatoms.get_potential_energy()

    res = re.match('(\w.*)_(\d{3}).*', file)
    ele = res.group(1)
    mir = res.group(2)
    
    num = 0
    ads = None
    res = re.match('.*n(\d+)_', file)
    if res:
        num = int(res.group(1))
        
    res = re.match('.*u(\d+).*', file)
    unit = int(res.group(1))

    res = re.match('.*no.*_(\w.)_n', file)
    if res:
        ads = res.group(1)

    ### calc area ###
    x = atoms.cell[0][0]
    y = atoms.cell[1][1]
    area = x*y

    ### calc adsorvates number ###
    tot = 0
    for atom in atoms:
        if atom.tag == 1:
            tot += 1

    ### get energies ###
    ene = atoms.get_potential_energy()
    bareene = bareatoms.get_potential_energy()
    totaladsene = ene - (COene*num + bareene)
    
    ### get adsorbate position info ###
    igroups, iposlis, rgroups, rposlis = getadsposinfo(file, 0)
    
    ### get COint info ###
    try:
        file_ = file[:-5] + '__.traj'
        atoms_ = query(file_, 'spacom')
        ene_ = atoms_.get_potential_energy()
    except:
        ene_ = 0
        
    ### get COlength info ###
    COlengthlis = []
    for i in range(len(atoms)):
        if atoms[i].symbol == 'C':
            Cpos = atoms[i].position
            Opos = atoms[i+1].position
            COlengthlis.append(np.linalg.norm(Cpos-Opos))
            
    dic['name'] = file
    dic['element'] = ele
    dic['mirror'] = mir
    dic['adsorbate'] = ads
    dic['unitlength'] = unit
    dic['surfatomnum'] = tot
    dic['area'] = area
    dic['Energy'] = ene
    dic['totaladsEnergy'] = totaladsene
    dic['aveadsEne/suratom'] = totaladsene/tot
    dic['numberofads'] = num
    dic['coverage'] = num/tot
    dic['density'] = num/area
    dic['igroups'] = igroups
    dic['iposlis'] = [list(item) for item in iposlis]
    dic['rgroups'] = rgroups
    dic['rposlis'] = [list(item) for item in rposlis]
    dic['ECO_Eint'] = ene_
    dic['Ediff'] = ene - ene_
    dic['COlengthlis'] = COlengthlis

    if num:
        dic['aveadsEne/ads'] = totaladsene/num
    else:
        dic['aveadsEne/ads'] = None
    maxdiff = getmaxdiff(file)

    if maxdiff > 1.5:
        dic['moved+'] = 'yes'
    else:
        dic['moved+'] = 'no'
    
    if igroups == rgroups:
        dic['moved'] = 'no'
    else:
        dic['moved'] = 'yes'
    
    for key in dic.keys():
        print('{}: {}'.format(key, dic[key]))
    
    return dic

In [6]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')
for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if '__' not in files[i]:
                print(i, files[i])

1 Ag_100_u1.traj
3 Ag_100_u1_no01_CO_n1_d3.traj
7 Ag_100_u1_no02_CO_n1_d3.traj
11 Ag_100_u1_no03_CO_n1_d3.traj
15 Ag_100_u2.traj
17 Ag_100_u2_no01_CO_n1_d9.traj
21 Ag_100_u2_no02_CO_n2_d3.traj
25 Ag_100_u2_no03_CO_n3_d3.traj
29 Ag_100_u2_no04_CO_n4_d3.traj
33 Ag_100_u2_no05_CO_n3_d3.traj
37 Ag_100_u2_no06_CO_n4_d3.traj
41 Ag_100_u2_no07_CO_n2_d6.traj
45 Ag_100_u2_no08_CO_n2_d4.traj
49 Ag_100_u2_no09_CO_n3_d3.traj
53 Ag_100_u2_no10_CO_n1_d9.traj
57 Ag_100_u2_no11_CO_n2_d3.traj
61 Ag_100_u2_no12_CO_n3_d3.traj
65 Ag_100_u2_no13_CO_n4_d3.traj
69 Ag_100_u2_no14_CO_n3_d3.traj
73 Ag_100_u2_no15_CO_n4_d3.traj
77 Ag_100_u2_no16_CO_n2_d6.traj
81 Ag_100_u2_no17_CO_n2_d4.traj
85 Ag_100_u2_no18_CO_n3_d3.traj
89 Ag_100_u2_no19_CO_n2_d3.traj
93 Ag_100_u2_no20_CO_n1_d9.traj
97 Ag_100_u2_no21_CO_n2_d3.traj
101 Ag_100_u2_no22_CO_n3_d3.traj
105 Ag_100_u2_no23_CO_n4_d3.traj
109 Ag_100_u2_no24_CO_n2_d6.traj
113 Ag_100_u3.traj
115 Ag_100_u3_no01_CO_n1_d15.traj
119 Ag_100_u3_no02_CO_n1_d15.traj
123 Ag_100_u3

In [7]:
makejson(files[3])

Symmetrically same structure found!
name: Ag_100_u1_no01_CO_n1_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -22.44298044
totaladsEnergy: 0.1705699100000011
aveadsEne/suratom: 0.1705699100000011
numberofads: 1
coverage: 1.0
density: 0.11267657015369789
igroups: [0]
iposlis: [[0.0, 0.0, 18.169597920855004]]
rgroups: [0]
rposlis: [[5.521685380174929e-07, 5.944197952180715e-07, 18.562673073888003]]
ECO_Eint: -14.2275679
Ediff: -8.215412539999999
COlengthlis: [1.1576383705845288]
aveadsEne/ads: 0.1705699100000011
moved+: no
moved: no


{'name': 'Ag_100_u1_no01_CO_n1_d3.traj',
 'element': 'Ag',
 'mirror': '100',
 'adsorbate': 'CO',
 'unitlength': 1,
 'surfatomnum': 1,
 'area': 8.874959529172191,
 'Energy': -22.44298044,
 'totaladsEnergy': 0.1705699100000011,
 'aveadsEne/suratom': 0.1705699100000011,
 'numberofads': 1,
 'coverage': 1.0,
 'density': 0.11267657015369789,
 'igroups': [0],
 'iposlis': [[0.0, 0.0, 18.169597920855004]],
 'rgroups': [0],
 'rposlis': [[5.521685380174929e-07,
   5.944197952180715e-07,
   18.562673073888003]],
 'ECO_Eint': -14.2275679,
 'Ediff': -8.215412539999999,
 'COlengthlis': [1.1576383705845288],
 'aveadsEne/ads': 0.1705699100000011,
 'moved+': 'no',
 'moved': 'no'}

In [8]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')

for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if 'CO.traj' not in files[i]:
                if '__' not in files[i]:
                    print(i, files[i])

                    post_data = makejson(files[i])
                    result = collection.insert_one(post_data)
                    print('One post: {0}\n'.format(result.inserted_id))

1 Ag_100_u1.traj
Symmetrically same structure found!
name: Ag_100_u1.traj
element: Ag
mirror: 100
adsorbate: 0
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -8.12811404
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
coverage: 0.0
density: 0.0
igroups: []
iposlis: []
rgroups: []
rposlis: []
ECO_Eint: 0
Ediff: -8.12811404
COlengthlis: []
aveadsEne/ads: None
moved+: no
moved: no
One post: 5d7c0ce224fcd5b4cb7e3364

3 Ag_100_u1_no01_CO_n1_d3.traj
Symmetrically same structure found!
name: Ag_100_u1_no01_CO_n1_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -22.44298044
totaladsEnergy: 0.1705699100000011
aveadsEne/suratom: 0.1705699100000011
numberofads: 1
coverage: 1.0
density: 0.11267657015369789
igroups: [0]
iposlis: [[0.0, 0.0, 18.169597920855004]]
rgroups: [0]
rposlis: [[5.521685380174929e-07, 5.944197952180715e-07, 18.562673073888003]]
ECO_Eint: -14.2275679
Ediff: -8.215412539999999
COlengthlis: [1.1576

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no05_CO_n3_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -75.57290444
totaladsEnergy: 0.343007340000014
aveadsEne/suratom: 0.0857518350000035
numberofads: 3
coverage: 0.75
density: 0.08450742761527343
igroups: [1, 0, 0]
iposlis: [[1.489543514736326, 2.979087029472652, 18.169597920855004], [2.979087029472652, 0.0, 18.169597920855004], [0.0, 0.0, 18.169597920855004]]
rgroups: [1, 0, 0]
rposlis: [[1.4896291924741782, 2.979090970553484, 18.10963622080176

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no11_CO_n2_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -61.15885916
totaladsEnergy: 0.27161631000000597
aveadsEne/suratom: 0.06790407750000149
numberofads: 2
coverage: 0.5
density: 0.056338285076848946
igroups: [1, 1]
iposlis: [[0.0, 4.468630544208981, 18.169597920855004], [2.979087029472652, 4.468630544208981, 18.169597920855004]]
rgroups: [1, 1]
rposlis: [[1.5814668199351402e-09, 4.468630486636436, 18.137054398109175], [2.979087028966683, 4.468630487288288, 18.137054400447923]]
ECO_Eint: -28.73474033
Ediff: -32.42411883
COlengthlis: [1.165508327671044, 1.1655083199031595]
aveadsEne/ads: 0.13580815500000298
moved+: no
moved: no
One post: 5d7c0cea24fcd5b4cb7e3373

61 Ag_100_u2_no12_CO_n3_d3.traj
Symmetrically same structure found!

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no17_CO_n2_d4.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -61.3662071
totaladsEnergy: 0.06426837000000774
aveadsEne/suratom: 0.016067092500001934
numberofads: 2
coverage: 0.5
density: 0.056338285076848946
igroups: [2, 1]
iposlis: [[4.468630544208981, 1.4895435147363276, 18.169597920855004], [2.979087029472652, 4.468630544208981, 18.169597920855004]]
rgroups: [1, 1]
rposlis: [[0.21538370791447148, 1.4891342937676266, 19.8401935958771], [2.8664120201891845, 4.469227463548521, 18.195114316239952]]
ECO_Eint: -28.96188277
Ediff: -32.404324329999994
COlengthlis: [1.1649811170727353, 4.829992125910998]
aveadsEne/ads: 0.03213418500000387
moved+: yes
moved: yes
One post: 5d7c0cee24fcd5b4cb7e3379

85 Ag_100_u2_no18_CO_n3_d3.traj
Symmetrically 

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no24_CO_n2_d6.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -61.43607326
totaladsEnergy: -0.005597789999995939
aveadsEne/suratom: -0.0013994474999989848
numberofads: 2
coverage: 0.5
density: 0.056338285076848946
igroups: [2, 2]
iposlis: [[4.468630544208981, 1.4895435147363276, 18.169597920855004], [1.489543514736326, 4.468630544208981, 18.169597920855004]]
rgroups: [2, 2]
rposlis: [[4.465774551705367, 1.4912931028913754, 20.117168970147276], [1.48693534

...

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Pt_111_u2_no22_CO_n3_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -128.40256009
totaladsEnergy: -2.0259515099999987
aveadsEne/suratom: -0.5064878774999997
numberofads: 3
coverage: 0.75
density: 0.108752834399702

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure